### 얼굴 필터 적용

In [1]:
import cv2
import mediapipe as mp
import numpy as np

# 람쥐 필터 이미지 (투명 PNG)
filter_img = cv2.imread('squirrel1.png', cv2.IMREAD_UNCHANGED)

# mediapipe 얼굴 검출 초기화
mp_face = mp.solutions.face_detection
mp_draw = mp.solutions.drawing_utils
face_detection = mp_face.FaceDetection(model_selection=0, min_detection_confidence=0.5)

# 필터 오버레이 함수
def overlay_filter(frame, filter_img, x, y, w, h):
    filter_resized = cv2.resize(filter_img, (w, h))

    for c in range(3):  # BGR 채널
        frame[y:y+h, x:x+w, c] = frame[y:y+h, x:x+w, c] * (1 - filter_resized[:, :, 3]/255.0) + \
                                 filter_resized[:, :, c] * (filter_resized[:, :, 3]/255.0)

# 웹캠 열기
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 좌우 반전
    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape

    # 얼굴 감지
    results = face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            x = int(bboxC.xmin * w)
            y = int(bboxC.ymin * h)
            box_w = int(bboxC.width * w)
            box_h = int(bboxC.height * h)

            # 얼굴 기준으로 필터 위치 조정 (이마 쪽으로 위로 이동)
            offset_y = int(box_h * 0.4)
            x1 = max(0, x - int(box_w * 0.2))
            y1 = max(0, y - offset_y)
            x2 = min(w, x1 + int(box_w * 1.4))
            y2 = min(h, y1 + int(box_h * 1.5))

            filter_w = x2 - x1
            filter_h = y2 - y1

            if filter_w > 0 and filter_h > 0:
                overlay_filter(frame, filter_img, x1, y1, filter_w, filter_h)

    cv2.imshow('Squirrel Filter 🐿️', frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\Users\SSAFY\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


#### 눈감으면 다른 가면

In [2]:
import cv2
import mediapipe as mp
import numpy as np

# 얼굴 필터 이미지 로드
filters = {
    "neutral": cv2.imread('neutral.png', cv2.IMREAD_UNCHANGED),
    "eyeclosed_neutral": cv2.imread('eyeclosed_neutral.png', cv2.IMREAD_UNCHANGED),
    "mouth_open": cv2.imread('mouth_open.png', cv2.IMREAD_UNCHANGED),
    "eyeclosed_mouthopen": cv2.imread('eyeclosed_mouthopen.png', cv2.IMREAD_UNCHANGED),
    "happy": cv2.imread('happy.png', cv2.IMREAD_UNCHANGED),
    "eyeclosed_happy": cv2.imread('eyeclosed_happy.png', cv2.IMREAD_UNCHANGED),
    "veryhappy": cv2.imread('veryhappy.png', cv2.IMREAD_UNCHANGED),
}

# Mediapipe 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_hands = mp.solutions.hands
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)

# EAR 계산 함수
def get_ear(landmarks, indices, w, h):
    pts = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in indices]
    A = np.linalg.norm(np.array(pts[1]) - np.array(pts[5]))
    B = np.linalg.norm(np.array(pts[2]) - np.array(pts[4]))
    C = np.linalg.norm(np.array(pts[0]) - np.array(pts[3]))
    return (A + B) / (2.0 * C)

# 입 벌림 판단 함수
def get_mouth_opening(landmarks, h):
    return abs(landmarks[13].y - landmarks[14].y) * h

# 미소 판단 함수
def is_smiling(landmarks, w, h):
    left = landmarks[61]
    right = landmarks[291]
    top = landmarks[13]
    bottom = landmarks[14]
    mouth_width = abs(right.x - left.x) * w
    mouth_height = abs(bottom.y - top.y) * h
    ratio = mouth_width / (mouth_height + 1e-6)
    return ratio > 2.2

# 손을 갈색으로 칠하는 함수
def draw_hand_mask(image, hand_landmarks, color=(60, 40, 20), alpha=0.5):
    h, w, _ = image.shape
    overlay = image.copy()
    for lm in hand_landmarks.landmark:
        x, y = int(lm.x * w), int(lm.y * h)
        cv2.circle(overlay, (x, y), 20, color, -1)
    cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0, image)

# 필터 오버레이 함수
def overlay_filter(frame, filter_img, x, y, w, h):
    filter_resized = cv2.resize(filter_img, (w, h))
    alpha = filter_resized[:, :, 3] / 255.0
    for c in range(3):
        frame[y:y+h, x:x+w, c] = (1 - alpha) * frame[y:y+h, x:x+w, c] + alpha * filter_resized[:, :, c]

# 웹캠 실행
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    face_result = face_mesh.process(rgb)
    hand_result = hands.process(rgb)

    # 손 갈색 처리
    if hand_result.multi_hand_landmarks:
        for hand in hand_result.multi_hand_landmarks:
            draw_hand_mask(frame, hand)

    # 얼굴 필터 처리
    if face_result.multi_face_landmarks:
        for landmarks in face_result.multi_face_landmarks:
            ear_l = get_ear(landmarks.landmark, [33, 160, 158, 133, 153, 144], w, h)
            ear_r = get_ear(landmarks.landmark, [362, 385, 387, 263, 373, 380], w, h)
            eye_closed = (ear_l + ear_r) / 2 < 0.2
            mouth_open = get_mouth_opening(landmarks.landmark, h) > 15
            smiling = is_smiling(landmarks.landmark, w, h)

            # 얼굴 위치 계산
            xs = [lm.x for lm in landmarks.landmark]
            ys = [lm.y for lm in landmarks.landmark]
            min_x, max_x = int(min(xs) * w), int(max(xs) * w)
            min_y, max_y = int(min(ys) * h), int(max(ys) * h)
            face_w, face_h = max_x - min_x, max_y - min_y
            cx = max(0, min(min_x - face_w // 10, w - 1))
            cy = max(0, min(min_y - face_h // 3, h - 1))
            cw = min(int(face_w * 1.2), w - cx)
            ch = min(int(face_h * 1.5), h - cy)

            # 표정에 맞는 키 선택
            if smiling and mouth_open:
                key = "veryhappy"
            elif smiling and eye_closed:
                key = "eyeclosed_happy"
            elif smiling:
                key = "happy"
            elif mouth_open and eye_closed:
                key = "eyeclosed_mouthopen"
            elif mouth_open:
                key = "mouth_open"
            elif eye_closed:
                key = "eyeclosed_neutral"
            else:
                key = "neutral"

            # 필터 오버레이
            overlay_filter(frame, filters[key], cx, cy, cw, ch)

    cv2.imshow("🐿️ Squirrel Vtuber", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)

mask_img = cv2.imread("happy.png", cv2.IMREAD_UNCHANGED)

def rotate_image(img, angle):
    h, w = img.shape[:2]
    center = (w//2, h//2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_LINEAR)

def overlay_image(bg, fg, x, y):
    h, w = fg.shape[:2]
    if x<0 or y<0 or x+w>bg.shape[1] or y+h>bg.shape[0]:
        return bg
    alpha = fg[:,:,3]/255.0
    for c in range(3):
        bg[y:y+h, x:x+w, c] = (1-alpha)*bg[y:y+h, x:x+w, c] + alpha*fg[:,:,c]
    return bg

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # 입력을 바로 반전
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(rgb)

    if result.multi_face_landmarks:
        lm = result.multi_face_landmarks[0].landmark
        h, w = frame.shape[:2]

        left_eye = lm[33]
        right_eye = lm[263]
        nose = lm[1]

        dx = (right_eye.x-left_eye.x)*w
        dy = (right_eye.y-left_eye.y)*h
        angle = -np.degrees(np.arctan2(dy, dx))

        eye_dist = np.sqrt(dx**2+dy**2)
        scale = eye_dist / mask_img.shape[1] * 2.2

        new_w = int(mask_img.shape[1]*scale)
        new_h = int(mask_img.shape[0]*scale)

        resized = cv2.resize(mask_img, (new_w, new_h))
        rotated = rotate_image(resized, angle)

        x = int(nose.x*w - new_w/2)
        y = int(nose.y*h - new_h/2)

        frame = overlay_image(frame, rotated, x, y)

    cv2.imshow("Mask", frame)  # 최종 flip 필요 없음
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
angle = -np.degrees(np.arctan2(dy, dx))



In [13]:
import cv2
import mediapipe as mp
import numpy as np

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)

# 사용할 가면 이미지 미리 로드
masks = {
    "neutral": cv2.imread("neutral.png", cv2.IMREAD_UNCHANGED),
    "eyeclosed_neutral": cv2.imread("eyeclosed_neutral.png", cv2.IMREAD_UNCHANGED),
    "happy": cv2.imread("happy.png", cv2.IMREAD_UNCHANGED),
    "eyeclosed_happy": cv2.imread("eyeclosed_happy.png", cv2.IMREAD_UNCHANGED),
    "mouthopen": cv2.imread("mouth_open.png", cv2.IMREAD_UNCHANGED),
    "eyeclosed_mouthopen": cv2.imread("eyeclosed_mouthopen.png", cv2.IMREAD_UNCHANGED),
}

# 눈 감김 비율 계산
def eye_aspect_ratio(lm, idx, w, h):
    p1, p2, p3, p4, p5, p6 = [lm[i] for i in idx]
    p1 = np.array([p1.x*w, p1.y*h])
    p4 = np.array([p4.x*w, p4.y*h])
    vertical1 = np.linalg.norm(np.array([p2.x*w, p2.y*h]) - np.array([p6.x*w, p6.y*h]))
    vertical2 = np.linalg.norm(np.array([p3.x*w, p3.y*h]) - np.array([p5.x*w, p5.y*h]))
    horizontal = np.linalg.norm(p1 - p4)
    return (vertical1 + vertical2) / (2.0 * horizontal)

# 회전
def rotate_image(img, angle):
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_LINEAR)

# 합성
def overlay_image(bg, fg, x, y):
    h, w = fg.shape[:2]
    if x < 0 or y < 0 or x + w > bg.shape[1] or y + h > bg.shape[0]:
        return bg
    alpha = fg[:, :, 3] / 255.0
    for c in range(3):
        bg[y:y+h, x:x+w, c] = (1-alpha) * bg[y:y+h, x:x+w, c] + alpha * fg[:, :, c]
    return bg

LEFT_EYE_IDX = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_IDX = [362, 385, 387, 263, 373, 380]

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(rgb)

    if result.multi_face_landmarks:
        lm = result.multi_face_landmarks[0].landmark
        h, w = frame.shape[:2]

        # 기본 landmark
        left_eye = lm[33]
        right_eye = lm[263]
        nose = lm[1]

        dx = (right_eye.x - left_eye.x) * w
        dy = (right_eye.y - left_eye.y) * h
        angle = -np.degrees(np.arctan2(dy, dx))

        # 얼굴 높이 계산 (이마~턱)
        forehead = lm[10]
        chin = lm[152]
        face_height = np.sqrt(((forehead.x - chin.x) * w) ** 2 +
                              ((forehead.y - chin.y) * h) ** 2)

        # 눈, 입 특징 계산
        eye_left = eye_aspect_ratio(lm, LEFT_EYE_IDX, w, h)
        eye_right = eye_aspect_ratio(lm, RIGHT_EYE_IDX, w, h)
        eye_avg = (eye_left + eye_right) / 2.0
        is_eye_closed = eye_avg < 0.22  # 값은 상황에 맞게 튜닝

        mouth_w = np.linalg.norm(np.array([lm[61].x*w, lm[61].y*h]) -
                                 np.array([lm[291].x*w, lm[291].y*h]))
        mouth_h = np.linalg.norm(np.array([lm[13].x*w, lm[13].y*h]) -
                                 np.array([lm[14].x*w, lm[14].y*h]))

        is_mouth_open = mouth_h > 18
        is_smiling = (mouth_w / mouth_h > 2.0) and not is_mouth_open

        # 가면 선택
        if is_mouth_open:
            mask_key = "eyeclosed_mouthopen" if is_eye_closed else "mouthopen"
        elif is_smiling:
            mask_key = "eyeclosed_happy" if is_eye_closed else "happy"
        else:
            mask_key = "eyeclosed_neutral" if is_eye_closed else "neutral"

        mask_img = masks[mask_key]

        # 얼굴 높이에 맞춰 스케일 조정
        scale = face_height / mask_img.shape[0] * 1.2
        new_w = int(mask_img.shape[1] * scale)
        new_h = int(mask_img.shape[0] * scale * 1.2)  # 세로만 1.2배

        resized = cv2.resize(mask_img, (new_w, new_h))
        rotated = rotate_image(resized, angle)

        # 코 기준 위치 (살짝 위로 올림)
        x = int(nose.x * w - new_w // 2)
        y = int(nose.y * h - new_h * 0.55)

        frame = overlay_image(frame, rotated, x, y)

    cv2.imshow("Mask", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
